In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q streamlit
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.8 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 2s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

In [3]:
!pip install pdfplumber -q
!pip install langchain_community -q
!pip install pypdf
!pip install chromadb -q
!pip install pinecone-client langchain openai pdfplumber
!pip install pinecone
!pip install tiktoken
!pip install langchain-openai
!pip install python-slugify

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting require

In [4]:
%%writefile app.py

import streamlit as st
import os
import time
import pandas as pd
import pdfplumber
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from google.colab import files
from pinecone import Pinecone, ServerlessSpec
from openai import OpenAI
from slugify import slugify

# OpenAI API 키 설정
openai_api_key = ""

# Pinecone API 설정
api_key = ""
environment = ""
index_name = ""

os.environ["OPENAI_API_KEY"] = openai_api_key

# Pinecone 클라이언트 초기화
pc = Pinecone(api_key=api_key)
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")  # 기본 OpenAI 임베딩 모델

# Pinecone 인덱스 생성
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # OpenAI text-embedding-ada-002 차원
        metric="cosine",
        spec=ServerlessSpec(
            cloud="awp",
            region="us-east1"
        )
    )

index = pc.Index(index_name)

# GPT 모델
def generate_with_gpt(prompt):
    try:
        prompt = prompt[:3000]
        response = OpenAI(api_key=openai_api_key).chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500,
            n=2,
            temperature=0.7
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"GPT 생성 중 오류 발생: {e}")
        return None

# 파일 업로드
def upload_file():
    uploaded = files.upload()  # 파일 업로드
    return list(uploaded.keys())[0]  # 업로드된 첫 번째 파일의 경로 반환

# 파일 형식 처리
def load_file(uploaded_file):
    ext = uploaded_file.name.split('.')[-1].lower()  # 파일 확장자 추출
    content = ""

    if ext == 'pdf':
        with pdfplumber.open(uploaded_file) as pdf:
            for page in pdf.pages:
                content += page.extract_text()
    elif ext == 'csv':
        df = pd.read_csv(uploaded_file)
        content = df.to_string()
    elif ext == 'txt':
        content = uploaded_file.read().decode('utf-8')
    elif ext == 'xlsx':
        df = pd.read_excel(uploaded_file)
        content = df.to_string()
    else:
        print(f"지원하지 않는 파일 형식입니다: {ext}")

    return content

# Pinecone에 이미 저장된 데이터 ID를 확인
def get_existing_ids(index):
    existing_ids = []
    try:
        # Pinecone에서 데이터 100개를 가져오기
        fetch_response = index.fetch(ids=[f"chunk_{i}" for i in range(100)])
        existing_ids = [item.id for item in fetch_response['vectors'].values()]
    except Exception as e:
        print(f"데이터 확인 중 오류 발생: {e}")
    return existing_ids

# 업로드할 데이터가 이미 Pinecone에 있는지 확인
def upload_data_if_not_exists(split_texts, index, file_name):
    # 기존 데이터 ID 가져오기
    existing_ids = get_existing_ids(index)

    for i, doc in enumerate(split_texts):
        # 파일명과 번호를 결합한 ID 생성
        chunk_id = f"{file_name}_{i}"

        if chunk_id not in existing_ids:
            try:
                time.sleep(0.1)  # Rate Limiting 방지
                vector = embedding_model.embed_query(doc.page_content)  # 텍스트 임베딩 생성
                index.upsert([(chunk_id, vector, {"source": chunk_id, "text": doc.page_content})])  # Pinecone에 업로드
                print(f"업로드된 ID: {chunk_id}")
            except Exception as e:
                print(f"chunk_{i}에서 오류 발생: {e}")
        else:
            print(f"ID {chunk_id}는 이미 Pinecone에 존재합니다. 업로드를 건너뜁니다.")

# 텍스트 분할
def split_text(content):
    text_splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=50)

    # 문자열을 Document 객체로 변환하여 반환
    documents = text_splitter.split_text(content)
    return [Document(page_content=doc) for doc in documents]  # Document 객체로 반환

# Pinecone에서 쿼리를 검색
def search_in_pinecone(query, index, embedding_model, top_k=3):
    try:
        # 쿼리 임베딩 생성
        query_vector = embedding_model.embed_query(query)

        # Pinecone 인덱스에서 검색
        results = index.query(
            vector=query_vector,
            top_k=top_k,
            include_metadata=True
        )
        return results
    except Exception as e:
        print(f"검색 중 오류 발생: {e}")
        return None

def main():
    st.title("문서 Q&A 시스템")

    # 채팅 기록 초기화
    if "messages" not in st.session_state:
        st.session_state.messages = []

    # 채팅 기록 표시
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

    # 사용자 입력 받기
    if prompt := st.chat_input("질문을 입력하세요:"):
        # 사용자 메시지 추가
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        # Pinecone에서 검색된 결과
        search_results = search_in_pinecone(prompt, index, embedding_model)

        if search_results:
            # 검색된 텍스트를 결합하여 GPT에게 전달할 프롬프트
            context = "\n".join([match.metadata['text'] for match in search_results.matches])
            gpt_prompt = f"다음 내용을 바탕으로 사용자의 질문에 답해주세요: {prompt}\n\n{context}"

            # GPT 모델을 사용해 답변 생성
            with st.chat_message("assistant"):
                with st.spinner("답변을 생성 중입니다..."):
                    response_gpt = generate_with_gpt(gpt_prompt)
                    if response_gpt:
                        st.markdown(response_gpt)
                        # 어시스턴트 메시지 추가
                        st.session_state.messages.append({"role": "assistant", "content": response_gpt})

                        st.markdown("### 참고한 페이지:")
                        for i, match in enumerate(search_results.matches[:3]):  # 상위 3개 결과

                            # 전체 텍스트를 토글 형식
                            with st.expander(f"내용 보기 ({i + 1})"):
                                preview_text = match.metadata.get('text', '')[:400]
                                source = match.metadata.get('source', '알 수 없음')
                                st.markdown(f"**출처:** {source}")
                                st.markdown(f"{preview_text}...")
                    else:
                        st.error("답변 생성 중 오류가 발생했습니다.")
        else:
            with st.chat_message("assistant"):
                st.warning("검색 결과가 없습니다.")
                st.session_state.messages.append({"role": "assistant", "content": "검색 결과가 없습니다."})

    # 파일 업로드 섹션
    st.sidebar.header("파일 업로드")
    uploaded_file = st.sidebar.file_uploader("파일을 업로드하세요", type=['pdf', 'csv', 'txt', 'xlsx'])

    if uploaded_file is not None:
        st.sidebar.write(f"업로드된 파일: {uploaded_file.name}")

        file_name = os.path.splitext(uploaded_file.name)[0]
        file_name = slugify(file_name)

        st.sidebar.write(f"영문 파일명: {file_name}")

        if st.sidebar.button("파일 처리 및 업로드"):
            with st.spinner("파일을 처리 중입니다..."):
                file_content = load_file(uploaded_file)
                split_texts = split_text(file_content)
                upload_data_if_not_exists(split_texts, index, file_name)
            st.sidebar.success("파일 처리 및 업로드가 완료되었습니다!")

if __name__ == "__main__":
    main()


Writing app.py


In [13]:
!streamlit run app.py &>/content/logs.txt &

In [14]:
import urllib
print("Password/Enpoint IP for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.82.4.126


In [15]:
!npx localtunnel --port 8501

⠙your url is: https://sharp-taxis-dig.loca.lt
^C
